# Exp13: Control Text Generation with a locally running LLM

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator

Configuration

In [3]:
MODEL="mistralai/Mistral-7B-Instruct-v0.2"

Load the model and generate three sentences (as indicated by the end of sequence tokens) and print the longest sentence. This re-ranking will be based on the grammar classifiers later.

In [6]:
model = AutoModelForCausalLM.from_pretrained(MODEL, device_map="auto", torch_dtype=torch.float16, cache_dir="/mnt/qb/work/meurers/mpb672/cache")
tokenizer = AutoTokenizer.from_pretrained(MODEL, cache_dir="/mnt/qb/work/meurers/mpb672/cache")

config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:00<00:00, 3.94MB/s]
model.safetensors.index.json: 100%|███████████████████████████████████████████████████████████████████████| 25.1k/25.1k [00:00<00:00, 58.5MB/s]
model-00001-of-00003.safetensors:   0%|                                                                            | 0.00/4.94G [00:00<?, ?B/s]
model-00001-of-00003.safetensors:   0%|▏                                                                  | 10.5M/4.94G [00:00<01:15, 65.8MB/s]
model-00001-of-00003.safetensors:   1%|▍                                                                   | 31.5M/4.94G [00:00<00:45, 108MB/s]
model-00001-of-00003.safetensors:   1%|▋                                                                   | 52.4M/4.94G [00:00<00:46, 106MB/s]
model-00001-of-00003.safetensors:   1%|█                                                                   | 73.4M/4.94G [00:00<00:44, 1

In [61]:
prompt = """<s>[INST] Continue the writing with language on CEFR level C2. [/INST] I've been meaning to write for ages"""
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

In [62]:
eos_chars = [".", "!", "?", "\n"]
max_len = 128
text = ""
generated_tokens = []
with torch.no_grad():
    for _ in range(max_len):
        next_token_logits = model(input_ids=inputs.input_ids).logits
        next_token_id = torch.argmax(next_token_logits[:, -1, :], dim=-1).item()

        generated_tokens.append(next_token_id)
        next_token = tokenizer.decode(next_token_id)
        if any(eos_char in next_token for eos_char in eos_chars):
            print(tokenizer.decode(generated_tokens, skip_special_tokens=True))
        
        inputs['input_ids'] =  torch.cat((inputs['input_ids'], torch.tensor([[next_token_id]]).to('cuda')), dim=1)

generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(generated_text)

, but my schedule has been rather hectic lately.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard. However, I've finally managed to carve out a few hours for myself, and I'm determined to make the most of it.
, but my schedule has been rather hectic lately. Between juggling work commitments, family obligations, and social engagements, it's been a challenge to find the time and energy to put pen to paper, or rather, fingers to keyboard. However, I've finally managed to carve out a few hours for myself, and I'm determined to make the most of it.

, but my schedule has been rather hec

In [57]:
tokenizer.decode(13)

'\n'

In [35]:
eos_tokens

[842, 1200, 1550]